In [24]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!',',','.']
data_file = open("intents.json").read()
intents = json.loads(data_file)
print(intents)

{'intents': [{'tag': 'Intro', 'patterns': ['привет', 'здравствуйте', 'приветствую', 'доброе утро', 'добрый вечер', 'добрый день', 'здарова', 'салам', 'ассаламуалейкум'], 'responses': ['Здравстуйте! Чем я могу вам помочь?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['до скорого', 'все, покедова', 'рад был с вами поболтать', 'спасибо за приятное общение, до встречи.', 'пока бот', 'давай', 'до скорой встречи', 'увидимся', 'всего доброго', 'удачи', 'до свидания', 'уагалейкум ассалам'], 'responses': ['Надеюсь я вам помог, До скорой встречи!'], 'context_set': ''}, {'tag': 'Contact', 'patterns': ['Контакты', 'Какой у вас инстаграмм', 'Можно ли узнать ваш номер', 'Какой у вас адрес', 'Где вы находитесь', 'инстаграм', 'whatsapp', 'ватсап', 'уатсап', 'instagram'], 'responses': ['Адрес: Каскелен, Жангозина 36.\nТелефон, whatsapp: 8-(700)-080-85-25 \nInstagram: @modern_edu_school'], 'context_set': ''}, {'tag': 'Stock', 'patterns': ['Какие у вас имеются акции', 'Какие у вас имеются скидки', 

In [25]:
# preprocessing the json data
# tokenization
nltk.download('punkt')
nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\77013\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\77013\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [26]:
# lemmatize, lower each word and remove duplicates

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('words.pkl','wb')) 
pickle.dump(classes,open('classes.pkl','wb'))

67 documents
8 classes ['Advantages', 'Aim', 'Contact', 'Courses', 'Exit', 'Intro', 'Price', 'Stock']
102 unique lemmatized words ['instagram', 'whatsapp', 'адрес', 'акции', 'английский', 'английского', 'ассалам', 'ассаламуалейкум', 'бот', 'был', 'вами', 'вас', 'ватсап', 'ваш', 'ваша', 'вашей', 'ваши', 'вечер', 'все', 'всего', 'встречи', 'вы', 'где', 'давай', 'день', 'до', 'доброго', 'доброе', 'добрый', 'дорого', 'за', 'здарова', 'здравствуйте', 'имеются', 'инстаграм', 'инстаграмм', 'какая', 'какие', 'какой', 'компании', 'контакты', 'ктл', 'курсах', 'курсы', 'ли', 'лист', 'лучше', 'математика', 'математики', 'миссия', 'можно', 'находитесь', 'ниш', 'номер', 'о', 'общение', 'план', 'планы', 'плюс', 'плюсы', 'поболтать', 'пока', 'покедова', 'почему', 'прайс', 'преимущества', 'преимущество', 'привет', 'приветствую', 'приятное', 'про', 'промокод', 'рад', 'расскажите', 'рфмш', 'с', 'салам', 'свидания', 'себе', 'скидки', 'сколько', 'скорого', 'скорой', 'спасибо', 'стоимость', 'стоят', 'у', 'у

In [27]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

Training data created


In [28]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/200
14/14 [==============================] - 0s 2ms/step - loss: 2.1007 - accuracy: 0.1343
Epoch 2/200
14/14 [==============================] - 0s 1ms/step - loss: 2.0573 - accuracy: 0.1791
Epoch 3/200
14/14 [==============================] - 0s 2ms/step - loss: 1.9904 - accuracy: 0.2388
Epoch 4/200
14/14 [==============================] - 0s 2ms/step - loss: 1.9761 - accuracy: 0.2537
Epoch 5/200
14/14 [==============================] - 0s 2ms/step - loss: 1.8805 - accuracy: 0.3134
Epoch 6/200
14/14 [==============================] - 0s 2ms/step - loss: 1.8907 - accuracy: 0.2239
Epoch 7/200
14/14 [==============================] - 0s 1ms/step - loss: 1.8298 - accuracy: 0.3134
Epoch 8/200
14/14 [==============================] - 0s 1ms/step - loss: 1.8353 - accuracy: 0.3134
Epoch 9/200
14/14 [==============================] - 0s 2ms/step - loss: 1.7239 - accuracy: 0.4030
Epoch 10/200
14/14 [==============================] - 0s 1ms/step - loss: 1.6201 - accuracy: 0.4776
Epoch 11/

In [29]:
# load the saved model file
model = load_model('chatbot.h5')
intents = json.loads(open("intents.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [30]:
def clean_up_sentence(sentence):

    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):

    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
               
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
   
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [31]:
# function to get the response from the model
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result





In [32]:
# function to start the chat bot which will continue till the user type 'end'

def bot_initialize():
    print("ЧатБот: Это Ерикбурден! Ваш персональный помощник.\n\n")
    while True:
        inp = str(input()).lower()
        if inp.lower()=="Закончить":
            break
        if inp.lower()== '' or inp.lower()== '*':
            print('Пожалуйста измените свой вопрос, я вас не понимаю(')
            print("-"*50)
        else:
            print(f"Бот: {chatbot_response(inp)}"+'\n')
            print("-"*50)

In [33]:
import requests
import json

class telegram_bot():
    def __init__(self):
        self.token = "5302765218:AAE_WU0FEEHCXIzF6aPnaYKkwgI2sG1OgJg"    #write your token here!
        self.url = f"https://api.telegram.org/bot{self.token}"

    def get_updates(self,offset=None):
        url = self.url+"/getUpdates?timeout=100"   # In 100 seconds if user input query then process that, use it as the read timeout from the server
        if offset:
            url = url+f"&offset={offset+1}"
        url_info = requests.get(url)
        return json.loads(url_info.content)

    def send_message(self,text,chat_id):
        url = self.url + f"/sendMessage?chat_id={chat_id}&text={text}"
        if text is not None:
            requests.get(url)

    def grab_token(self):
        return tokens

In [34]:
tbot = telegram_bot()

update_id = None

# function to predict the class and get the response

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res    
    
       
while True:
    print("...")
    updates = tbot.get_updates(offset=update_id)
    updates = updates['result']
    print(updates)
    if updates:
        for item in updates:
            update_id = item["update_id"]
            print(update_id)
            try:
                message = item["message"]["text"]
                print(message)
            except:
                message = None
            from_ = item["message"]["from"]["id"]
            print(from_)

            res = chatbot_response(message)
            tbot.send_message(res,from_)

...
[{'update_id': 838796123, 'message': {'message_id': 183, 'from': {'id': 458511059, 'is_bot': False, 'first_name': 'Bilal', 'username': 'Bilalalladin', 'language_code': 'ru'}, 'chat': {'id': 458511059, 'first_name': 'Bilal', 'username': 'Bilalalladin', 'type': 'private'}, 'date': 1652458895, 'text': 'Контакты'}}]
838796123
Контакты
458511059
...
[{'update_id': 838796124, 'message': {'message_id': 185, 'from': {'id': 458511059, 'is_bot': False, 'first_name': 'Bilal', 'username': 'Bilalalladin', 'language_code': 'ru'}, 'chat': {'id': 458511059, 'first_name': 'Bilal', 'username': 'Bilalalladin', 'type': 'private'}, 'date': 1652458905, 'text': 'какие у вас цены'}}]
838796124
какие у вас цены
458511059
...
[{'update_id': 838796125, 'message': {'message_id': 187, 'from': {'id': 458511059, 'is_bot': False, 'first_name': 'Bilal', 'username': 'Bilalalladin', 'language_code': 'ru'}, 'chat': {'id': 458511059, 'first_name': 'Bilal', 'username': 'Bilalalladin', 'type': 'private'}, 'date': 165245